In [120]:
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import random
import ast

# prendre une cartede plus haute resolution pour plus de pays
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) 
climate_raster = 'map.tif'


In [93]:
# Define the climate legend as a dictionary mapping RGB values to climate classes
climate_legend = {
    (0, 0, 255): "Af",      # Tropical, rainforest
    (0, 120, 255): "Am",    # Tropical, monsoon
    (70, 170, 250): "Aw",   # Tropical, savannah
    (255, 0, 0): "BWh",     # Arid, desert, hot
    (255, 150, 150): "BWk", # Arid, desert, cold
    (245, 165, 0): "BSh",   # Arid, steppe, hot
    (255, 220, 100): "BSk", # Arid, steppe, cold
    (255, 255, 0): "Csa",   # Temperate, dry summer, hot summer
    (200, 200, 0): "Csb",   # Temperate, dry summer, warm summer
    (150, 150, 0): "Csc",   # Temperate, dry summer, cold summer
    (150, 255, 150): "Cwa", # Temperate, dry winter, hot summer
    (100, 200, 100): "Cwb", # Temperate, dry winter, warm summer
    (50, 150, 50): "Cwc",   # Temperate, dry winter, cold summer
    (200, 255, 80): "Cfa",  # Temperate, no dry season, hot summer
    (100, 255, 80): "Cfb",  # Temperate, no dry season, warm summer
    (50, 200, 0): "Cfc",    # Temperate, no dry season, cold summer
    (255, 0, 255): "Dsa",   # Cold, dry summer, hot summer
    (200, 0, 200): "Dsb",   # Cold, dry summer, warm summer
    (150, 50, 150): "Dsc",  # Cold, dry summer, cold summer
    (150, 100, 150): "Dsd", # Cold, dry summer, very cold winter
    (170, 175, 255): "Dwa", # Cold, dry winter, hot summer
    (90, 120, 220): "Dwb",  # Cold, dry winter, warm summer
    (75, 80, 180): "Dwc",   # Cold, dry winter, cold summer
    (50, 0, 135): "Dwd",    # Cold, dry winter, very cold winter
    (0, 255, 255): "Dfa",   # Cold, no dry season, hot summer
    (55, 200, 255): "Dfb",  # Cold, no dry season, warm summer
    (0, 125, 125): "Dfc",   # Cold, no dry season, cold summer
    (0, 70, 95): "Dfd",     # Cold, no dry season, very cold winter
    (178, 178, 178): "ET",  # Polar, tundra
    (102, 102, 102): "EF"   # Polar, frost
}


In [100]:
import rasterio
from shapely.geometry import Point
import random

# Function to check if a point is within the bounds of the climate raster
def is_within_raster(lat, lon, raster_bounds):
    min_lon, min_lat, max_lon, max_lat = raster_bounds
    return min_lon <= lon <= max_lon and min_lat <= lat <= max_lat

# Function to check if a point corresponds to valid data in the raster (not NoData)
def has_valid_data(lat, lon, raster, transform, nodata_value):
    # Convert lat/lon to row/col in the raster
    col, row = ~transform * (lon, lat)
    row, col = int(row), int(col)

    # Ensure the coordinates are within the raster's bounds
    if row < 0 or col < 0 or row >= raster.shape[1] or col >= raster.shape[2]:
        return False  # Out of bounds

    # Check the raster value at this row/col to ensure it isn't NoData
    pixel_value = raster[:, row, col]
    if pixel_value[0] == nodata_value:  # Compare with the nodata value
        return False  # NoData point

    return True

# Function to generate 3 valid points (centroid + 2 valid points) per country
def generate_valid_points_for_country(country_geom, climate_raster_file, max_attempts=100):
    points = []

    # Open the climate GeoTIFF file to get bounds, transformation data, and nodata value
    with rasterio.open(climate_raster_file) as src:
        transform = src.transform
        raster_bounds = src.bounds
        climate_data = src.read()  # Read all bands (for RGB values)
        nodata_value = src.nodata  # Access the nodata value from the file

        # Step 1: Add the centroid of the country as the first point
        centroid = country_geom.centroid
        if is_within_raster(centroid.y, centroid.x, raster_bounds) and has_valid_data(centroid.y, centroid.x, climate_data, transform, nodata_value):
            points.append((centroid.y, centroid.x))  # Latitude, Longitude
        else:
            # If the centroid is out of bounds or NoData, generate a valid point inside the country's boundary
            points.append(generate_random_point_within_country(country_geom, climate_data, transform, raster_bounds, nodata_value))

        # Step 2: Generate two additional valid points inside the country and within the GeoTIFF map
        for _ in range(2):
            point = generate_random_point_within_country(country_geom, climate_data, transform, raster_bounds, nodata_value, max_attempts)
            if point is not None:
                points.append(point)
            else:
                points.append(('No valid point found', ''))  # Fallback if no valid points found
    
    return points

# Function to generate a valid random point inside the country's boundaries and within the GeoTIFF map bounds
def generate_random_point_within_country(country_geom, climate_data, transform, raster_bounds, nodata_value, max_attempts=100):
    minx, miny, maxx, maxy = country_geom.bounds
    attempts = 0

    while attempts < max_attempts:
        # Generate a random lat/lon point within the bounding box of the country
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        
        # Check if the point is inside the country and within the bounds of the raster, and has valid data
        if country_geom.contains(random_point) and is_within_raster(random_point.y, random_point.x, raster_bounds):
            if has_valid_data(random_point.y, random_point.x, climate_data, transform, nodata_value):
                return (random_point.y, random_point.x)  # Return valid latitude and longitude
        
        attempts += 1

    # If no valid point is found after max_attempts, return None (or handle as needed)
    return None

country_points = {}

# Iterate through each country and generate points
for _, row in world.iterrows():
    country_name = row['name']
    country_geom = row['geometry']
    points = generate_valid_points_for_country(country_geom, climate_raster)
    country_points[country_name] = points


In [ ]:
del country_points['Fiji']

country_points

In [105]:
def get_climate(lat, lon, raster, transform, raster_bounds, legend):
    # Convert lat/lon to float to ensure proper comparison
    lat = float(lat)
    lon = float(lon)
    
    min_lon, min_lat, max_lon, max_lat = raster_bounds
    # Check if lat/lon is within raster bounds
    if min_lon <= lon <= max_lon and min_lat <= lat <= max_lat:
        # Convert lat/lon to row/col in the raster
        row, col = ~transform * (lon, lat)  # lon, lat order for transformation
        row, col = int(row), int(col)
        
        # Ensure row/col are within the valid range of the raster's grid
        if 0 <= row < raster.shape[0] and 0 <= col < raster.shape[1]:
            climate_value = raster[row, col]  # Get the numeric climate classification
            
            # Map the numeric value to its Köppen-Geiger class using the legend
            #climate_type = legend.get(climate_value, 'Unknown Climate')
            climate_type = climate_value
        else:
            climate_type = 'No Climate Data'
    else:
        climate_type = 'No Climate Data'  # Handle out-of-bounds case
    
    return climate_type
    
def assign_climate_to_countries(country_dict, climate_raster_file, legend):
    country_climate_map = {}

    with rasterio.open(climate_raster_file) as src:
        climate_data = src.read(1)  # Read the first band of the GeoTIFF
        transform = src.transform
        raster_bounds = src.bounds

        for country, points in country_dict.items():
            climate_types = []
            for point in points:
                if isinstance(point, tuple) and len(point) == 2:  # Ensure the point is a tuple with latitude and longitude
                    lat, lon = point
                    # Ensure lat and lon are floats
                    lat = float(lat)
                    lon = float(lon)
                    # Get the climate type for each point using the legend
                    climate_type = get_climate(lat, lon, climate_data, transform, raster_bounds, legend)
                    climate_types.append(climate_type)
                else:
                    climate_types.append('Invalid Point Data')  # Handle invalid points
            country_climate_map[country] = climate_types

    return country_climate_map


country_climate_map = assign_climate_to_countries(country_points, climate_raster, climate_legend)

# Step 7: Display the results
for country, climates in country_climate_map.items():
    print(f"Country: {country}")
    for i, climate in enumerate(climates):
        print(f"  Point {i+1}: Climate Type: {climate}")


IndexError: index 2147 is out of bounds for axis 0 with size 1800

In [8]:
import geopandas as gpd

# Path to your shapefile
shapefile_path = 'map/c1976_2000.shp'

# Load the shapefile using GeoPandas
climate_shapefile = gpd.read_file(shapefile_path)

# Step 1: Display the column names to understand what data is available
print("Column Names in the Shapefile:")
print(climate_shapefile.columns)

# Step 2: Check the first few rows to inspect the structure of the data
print("\nFirst few rows of the Shapefile:")
print(climate_shapefile.head())

# Step 3: Check the data types of each column
print("\nData types of each column:")
print(climate_shapefile.dtypes)

# Step 4: (Optional) Describe the numeric columns, if any, to get an overview of the data
print("\nSummary of numeric columns:")
print(climate_shapefile.describe())

# Step 5: (Optional) If you expect a geometry column, check the geometry of the shapefile
print("\nGeometry column (geometries present in the shapefile):")
print(climate_shapefile.geometry)


Column Names in the Shapefile:
Index(['ID', 'GRIDCODE', 'geometry'], dtype='object')

First few rows of the Shapefile:
   ID  GRIDCODE                                           geometry
0   1        62  POLYGON ((-37.50000 83.50000, -38.00000 83.500...
1   2        62  POLYGON ((-29.50000 83.50000, -37.00000 83.500...
2   3        62  POLYGON ((-46.00000 83.00000, -46.00000 83.500...
3   4        62  POLYGON ((-42.50000 83.50000, -39.50000 83.500...
4   5        61  POLYGON ((55.50000 81.00000, 55.50000 81.50000...

Data types of each column:
ID             int64
GRIDCODE       int64
geometry    geometry
dtype: object

Summary of numeric columns:
                ID     GRIDCODE
count  2259.000000  2259.000000
mean   1130.000000    34.093404
std     652.261451    14.642194
min       1.000000    11.000000
25%     565.500000    26.000000
50%    1130.000000    34.000000
75%    1694.500000    43.000000
max    2259.000000    62.000000

Geometry column (geometries present in the shapefile):
0

In [11]:
import geopandas as gpd

# Path to your shapefile
shapefile_path = 'map/c1976_2000.shp'

# Load the shapefile using GeoPandas
climate_shapefile = gpd.read_file(shapefile_path)

# Display the first few rows to understand the structure
print(climate_shapefile.head())

# Since there's no 'country' column, you need to intersect the geometries with a world borders dataset
# You can use a world borders dataset provided by GeoPandas, like "naturalearth_lowres"
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Ensure both datasets use the same coordinate reference system (CRS)
# Reproject the world data to match the shapefile CRS if needed
if climate_shapefile.crs != world.crs:
    world = world.to_crs(climate_shapefile.crs)

# Perform a spatial join to map climate polygons to countries
# This will give each climate polygon a corresponding country based on spatial intersection
climate_with_countries = gpd.sjoin(climate_shapefile, world, how="inner", op="intersects")

# Now group the data by country and get the unique climate classifications (GRIDCODEs) for each country
climate_per_country = climate_with_countries.groupby('name')['GRIDCODE'].unique()

# Convert to a dictionary for easier access
climate_per_country_dict = climate_per_country.to_dict()

# Display the climate types for each country
for country, climates in climate_per_country_dict.items():
    print(f"Country: {country}, Climate Types: {climates}")


   ID  GRIDCODE                                           geometry
0   1        62  POLYGON ((-37.50000 83.50000, -38.00000 83.500...
1   2        62  POLYGON ((-29.50000 83.50000, -37.00000 83.500...
2   3        62  POLYGON ((-46.00000 83.00000, -46.00000 83.500...
3   4        62  POLYGON ((-42.50000 83.50000, -39.50000 83.500...
4   5        61  POLYGON ((55.50000 81.00000, 55.50000 81.50000...
Country: Afghanistan, Climate Types: [45 47 27 21 34 46 26 62 43 35 32 31 22]
Country: Albania, Climate Types: [42 32 35 34]
Country: Algeria, Climate Types: [27 34 21 26 22]
Country: Angola, Climate Types: [22 27 14 37 38 21]
Country: Antarctica, Climate Types: [61]
Country: Argentina, Climate Types: [38 26 39 27 22 36 21 37 62 35 32 31 33]
Country: Armenia, Climate Types: [41 31 42 26 45]
Country: Australia, Climate Types: [12 14 31 11 37 27 21 32 22 34 35 26]
Country: Austria, Climate Types: [43 42 62 32]
Country: Azerbaijan, Climate Types: [32 42 43 31 26 41 34 45]
Country: Bahamas, Clim

In [15]:
 df =pd.DataFrame([climate_per_country_dict])

In [19]:
df.transpose().to_csv('climate.csv')

In [128]:
df = pd.read_csv('climate.csv')
df.rename(columns={'Unnamed: 0': 'country'}, inplace=True)
df.rename(columns={'0': 'climate'}, inplace=True)



In [104]:
df

,country,climate
0,Afghanistan,[45 47 27 21 34 46 26 62 43 35 32 31 22]
1,Albania,[42 32 35 34]
2,Algeria,[27 34 21 26 22]
3,Angola,[22 27 14 37 38 21]
4,Antarctica,[61]
...,...,...
171,W. Sahara,[22]
172,Yemen,[21 27 26 22]
173,Zambia,[38 14 27 37]
174,Zimbabwe,[27 14 37 38 22]


In [131]:
import pandas as pd
import numpy as np
df = pd.read_csv('climate.csv')
df.rename(columns={'Unnamed: 0': 'country'}, inplace=True)
df.rename(columns={'0': 'climate'}, inplace=True)


def climate_to_continuous(koppen_code):
    temperature_map = {
        11: 1.0,  # Af
        12: 0.9,  # Am
        13: 0.8,  # As
        14: 0.7,  # Aw
        21: 0.6,  # BWk
        22: 0.5,  # BWh
        26: 0.4,  # BSk
        27: 0.3,  # BSh
        31: 0.6,  # Cfa
        32: 0.5,  # Cfb
        33: 0.4,  # Cfc
        34: 0.7,  # Csa
        35: 0.5,  # Csb
        36: 0.4,  # Csc
        37: 0.5,  # Cwa
        38: 0.4,  # Cwb
        39: 0.3,  # Cwc
        41: 0.6,  # Dfa
        42: 0.5,  # Dfb
        43: 0.4,  # Dfc
        44: 0.3,  # Dfd
        45: 0.4,  # Dsa
        46: 0.4,  # Dsb
        47: 0.4,  # Dsc
        48: 0.4,  # Dsd
        49: 0.3,  # Dwa
        50: 0.2,  # Dwb
        51: 0.1,  # Dwc
        52: 0.1,  # Dwd
        61: 0.1,  # EF
        62: 0.1   # ET
    }
    
    precipitation_map = {
        11: 1.0,  # Af
        12: 0.9,  # Am
        13: 0.9,  # As
        14: 0.7,  # Aw
        21: 0.1,  # BWk
        22: 0.2,  # BWh
        26: 0.3,  # BSk
        27: 0.4,  # BSh
        31: 0.7,  # Cfa
        32: 0.6,  # Cfb
        33: 0.5,  # Cfc
        34: 0.8,  # Csa
        35: 0.6,  # Csb
        36: 0.5,  # Csc
        37: 0.7,  # Cwa
        38: 0.8,  # Cwb
        39: 0.6,  # Cwc
        41: 0.5,  # Dfa
        42: 0.4,  # Dfb
        43: 0.3,  # Dfc
        44: 0.2,  # Dfd
        45: 0.4,  # Dsa
        46: 0.4,  # Dsb
        47: 0.4,  # Dsc
        48: 0.4,  # Dsd
        49: 0.3,  # Dwa
        50: 0.2,  # Dwb
        51: 0.1,  # Dwc
        52: 0.1,  # Dwd
        61: 0.1,  # EF
        62: 0.1   # ET
    }
    
    seasonality_map = {
        11: 1.0,  # Af
        12: 0.8,  # Am
        13: 0.7,  # As
        14: 0.6,  # Aw
        21: 0.1,  # BWk
        22: 0.1,  # BWh
        26: 0.2,  # BSk
        27: 0.3,  # BSh
        31: 0.6,  # Cfa
        32: 0.5,  # Cfb
        33: 0.4,  # Cfc
        34: 0.5,  # Csa
        35: 0.5,  # Csb
        36: 0.4,  # Csc
        37: 0.5,  # Cwa
        38: 0.4,  # Cwb
        39: 0.3,  # Cwc
        41: 0.5,  # Dfa
        42: 0.4,  # Dfb
        43: 0.3,  # Dfc
        44: 0.3,  # Dfd
        45: 0.4,  # Dsa
        46: 0.4,  # Dsb
        47: 0.4,  # Dsc
        48: 0.4,  # Dsd
        49: 0.3,  # Dwa
        50: 0.2,  # Dwb
        51: 0.1,  # Dwc
        52: 0.1,  # Dwd
        61: 0.1,  # EF
        62: 0.1   # ET
    }
    
    aridity_map = {
        11: 0.1,  # Af
        12: 0.2,  # Am
        13: 0.2,  # As
        14: 0.3,  # Aw
        21: 0.9,  # BWk
        22: 0.8,  # BWh
        26: 0.6,  # BSk
        27: 0.5,  # BSh
        31: 0.5,  # Cfa
        32: 0.5,  # Cfb
        33: 0.4,  # Cfc
        34: 0.4,  # Csa
        35: 0.5,  # Csb
        36: 0.4,  # Csc
        37: 0.3,  # Cwa
        38: 0.2,  # Cwb
        39: 0.2,  # Cwc
        41: 0.3,  # Dfa
        42: 0.4,  # Dfb
        43: 0.4,  # Dfc
        44: 0.3,  # Dfd
        45: 0.4,  # Dsa
        46: 0.4,  # Dsb
        47: 0.4,  # Dsc
        48: 0.4,  # Dsd
        49: 0.3,  # Dwa
        50: 0.3,  # Dwb
        51: 0.1,  # Dwc
        52: 0.1,  # Dwd
        61: 0.1,  # EF
        62: 0.1   # ET
    }
    
    # Convert the code into continuous values using the mappings above
    T = temperature_map.get(koppen_code, np.nan)
    P = precipitation_map.get(koppen_code, np.nan)
    S = seasonality_map.get(koppen_code, np.nan)
    A = aridity_map.get(koppen_code, np.nan)
    
    return {
        'Temperature': T,
        'Precipitation': P,
        'Seasonality': S,
        'Aridity': A
    }

def calculate_average_continuous(climate_codes):
    temperature_list = []
    precipitation_list = []
    seasonality_list = []
    aridity_list = []
    
    for code in climate_codes:
        continuous_values = climate_to_continuous(code)
        
        if not np.isnan(continuous_values['Temperature']):
            temperature_list.append(continuous_values['Temperature'])
        if not np.isnan(continuous_values['Precipitation']):
            precipitation_list.append(continuous_values['Precipitation'])
        if not np.isnan(continuous_values['Seasonality']):
            seasonality_list.append(continuous_values['Seasonality'])
        if not np.isnan(continuous_values['Aridity']):
            aridity_list.append(continuous_values['Aridity'])
    
    avg_temperature = np.nan if not temperature_list else sum(temperature_list) / len(temperature_list)
    avg_precipitation = np.nan if not precipitation_list else sum(precipitation_list) / len(precipitation_list)
    avg_seasonality = np.nan if not seasonality_list else sum(seasonality_list) / len(seasonality_list)
    avg_aridity = np.nan if not aridity_list else sum(aridity_list) / len(aridity_list)
    
    return {
        'Avg_Temperature': avg_temperature,
        'Avg_Precipitation': avg_precipitation,
        'Avg_Seasonality': avg_seasonality,
        'Avg_Aridity': avg_aridity
    }

df['climate'] = df['climate'].apply(lambda x: list(map(int, ast.literal_eval(x.replace(' ', ',')))))
df['Continuous_Climate'] = df['climate'].apply(calculate_average_continuous)

df_expanded = pd.concat([df['country'], df['Continuous_Climate'].apply(pd.Series)], axis=1)

df_expanded

In [133]:
df_expanded.to_csv('data.csv')